In [29]:
import torch
from torch import nn
import numpy as np
import random
from collections import deque
from itertools import count
import torch.nn.functional as F
from mdp import TradeExecutionEnv, DiscreteTradeSizeWrapper, RelativeTradeSizeWrapper
from tensorboardX import SummaryWriter
from torch.distributions import Categorical


SEED = 42
HORIZON = 5 * 12 * 2
UNITS_TO_SELL = 64
BATCH_SIZE = 32
EPOCHS = 300
TASK_BATCH_SIZE = 12
EPSILON = 0.1
GAMMA = 0.99

env = TradeExecutionEnv()

trade_sizes = {
  i: i*2 for i in range(33)
}
env = DiscreteTradeSizeWrapper(env, trade_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class CategoricalPolicy(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim=32) -> None:
        super().__init__()
        #self.rnn = nn.LSTM(num_states, hidden_dim, 1, batch_first=True)
        self.fc1 = nn.Linear(num_states, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_actions)
        
    def forward(self, x):
        #x, _ = self.rnn(x)
        x = F.relu(self.fc1(x[:, -1, :]))
        x = F.softmax(self.fc2(x), dim=1)
        return x

    def select_action(self, state):
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [30]:
def parse_state(state):
    data = torch.FloatTensor(np.stack([
        state["low"].to_numpy(),
        state["high"].to_numpy(),
        state["close"].to_numpy(),
        state["open"].to_numpy(),
        state["volume"].to_numpy(),
    ])).T
    return torch.concat([
        data,
        torch.repeat_interleave(torch.FloatTensor([[state["units_sold"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["cost_basis"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["steps_left"]]]), 6, 0),
    ], dim=1).unsqueeze(0)

In [31]:
def sample_rollouts(policy, env, num_rollouts, seed, test=False):
    trajs = []
    for _ in range(num_rollouts):
        tau = []
        state = env.reset(UNITS_TO_SELL, HORIZON, seed=seed, test=test)
        state = parse_state(state)
        done = False
        while not done:
            with torch.no_grad():
                action, _ = policy.select_action(state)
            next_state, reward, done, _, _ = env.step(action)
            next_state = parse_state(next_state)
            tau.append((state, action, reward))
            state = next_state
        states, actions, rewards = zip(*tau)
        states = torch.cat(states)
        actions = torch.tensor([actions])
        rewards = torch.tensor([rewards])
        trajs.append((states, actions, rewards))
    return trajs

In [32]:
def R_tau(r, gamma):
    gammas = torch.tensor([gamma**i for i in range(len(r))])
    return torch.sum(gammas * r)

def avg_batch_rewards(trajs, gamma):
    return torch.mean(torch.stack([R_tau(r, gamma) for _, _, r in trajs]))

def grad_log_pi(policy, states, actions):
    logits = policy(states)
    log_probs = torch.log(logits)
    log_probs = log_probs.gather(1,actions).T.flatten()
    g = [torch.autograd.grad(log_p, policy.parameters(), retain_graph=True) for log_p in log_probs]
    return g

In [33]:
def grad_log_tau(policy, tau):
  g = grad_log_pi(policy, tau[0], tau[1])
  stacked_g = [torch.stack(g_).view(len(tau[0]), -1) for g_ in zip(*g)]
  return torch.cat(stacked_g, axis=1).sum(axis=0)

def grad_U_tau(policy, tau, gamma):
  return grad_log_tau(policy, tau) * R_tau(tau[2], gamma)

def flatten_params(policy):
    return torch.cat([p.view(-1) for p in policy.parameters()])

In [34]:
policy = CategoricalPolicy(len(env.observation_space), len(trade_sizes)).to(device)

train_rewards = []
for e in range(EPOCHS):
  meta_g = torch.zeros_like(flatten_params(policy))
  task_rewards = []
  for i in range(TASK_BATCH_SIZE):
    trajs = sample_rollouts(policy, env, BATCH_SIZE, seed=SEED+i)
    u = torch.stack([grad_U_tau(policy, tau, GAMMA) for tau in trajs]).mean(axis=0)
    g = u * torch.sqrt(2 * EPSILON / (u @ u))
    n = 0
    new_policy = CategoricalPolicy(len(env.observation_space), len(trade_sizes)).to(device)
    new_policy.load_state_dict(policy.state_dict().copy())
    for i, p in enumerate(new_policy.parameters()):
      num_elements = p.numel()
      p.data += g[n:n+num_elements].view(p.shape)
      n += num_elements
    new_task_trajs = sample_rollouts(new_policy, env, BATCH_SIZE, seed=SEED+i)
    u = torch.stack([grad_U_tau(new_policy, tau, GAMMA) for tau in new_task_trajs]).mean(axis=0)
    meta_g += u * torch.sqrt(2 * EPSILON / (u @ u))
    policy.zero_grad()
    task_rewards.append(avg_batch_rewards(trajs, GAMMA))

  n = 0
  meta_g /= TASK_BATCH_SIZE
  for i, p in enumerate(policy.parameters()):
      num_elements = p.numel()
      p.data += g[n:n+num_elements].view(p.shape)
      n += num_elements
  train_rewards.append(np.mean(task_rewards))
  print(f"iter {e}: {np.mean(task_rewards)}")

iter 0: -9.368296662053682
iter 1: -9.329198349146735
iter 2: -9.458551085629361
iter 3: -9.549206685729429
iter 4: -9.283447218156303
iter 5: -9.29368610251781
iter 6: -9.381631244994194
iter 7: -9.165805880847262
iter 8: -9.106651755293452
iter 9: -9.21265914005159
iter 10: -9.432162557908967
iter 11: -9.111351823542876
iter 12: -9.388177804165698
iter 13: -9.266009650288444
iter 14: -9.005518068005193
iter 15: -8.794427528731896
iter 16: -8.709202852143234
iter 17: -8.440751184759847
iter 18: -6.95034133388306
iter 19: -5.773388441200328
iter 20: -4.3430476922461905
iter 21: -5.597221749120515
iter 22: -4.664323706827207
iter 23: -6.0196175999529915
iter 24: -4.659532773088661
iter 25: -6.076508908439494
iter 26: -4.348726667283899
iter 27: -3.5356501610186606
iter 28: -5.03045799977995
iter 29: -3.7847573372851016
iter 30: -5.263198670645856
iter 31: -3.9379056380048367
iter 32: -5.529952809384628
iter 33: -4.006970698657757
iter 34: -3.212663001408879
iter 35: -3.890705959391283
i

In [38]:
EVAL_EPOCHS = 50
K = 2

same_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    policy.load_state_dict(original_params)
    k_shot(policy, K, seed=SEED)
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    same_mdp_eval_rewards.append(episode_rewards)
print(f"Avg same mdp eval reward: {np.mean(same_mdp_eval_rewards)}")

ValueError: Expected parameter probs (Tensor of shape (1, 33)) of distribution Categorical(probs: torch.Size([1, 33])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [17]:
train_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    policy.load_state_dict(original_params)
    k_shot(policy, K, seed=(BATCH_SIZE * EPOCHS)+e)
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=(BATCH_SIZE * EPOCHS)+e)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    train_mdp_eval_rewards.append(episode_rewards)
print(f"Avg train mdp eval reward: {np.mean(train_mdp_eval_rewards)}")

Avg train mdp eval reward: -10.0


In [18]:
test_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    policy.load_state_dict(original_params)
    k_shot(env, policy, K, seed=SEED+e, test=True)
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED+e, test=True)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    test_mdp_eval_rewards.append(episode_rewards)
print(f"Avg test mdp eval reward: {np.mean(test_mdp_eval_rewards)}")

Avg test mdp eval reward: -10.0


In [19]:
import json

EXP_NAME = "maml_restricted_policy_update_fine_tune_mdp_train.json"
with open(f"./results/{EXP_NAME}", "w+") as f:
    json.dump({
        "train_rewards": train_rewards,
        "same_mdp_eval_rewards": same_mdp_eval_rewards,
        "train_mdp_eval_rewards": train_mdp_eval_rewards,
        "test_mdp_eval_rewards": test_mdp_eval_rewards
    }, f)

In [36]:
original_params = policy.state_dict().copy()

In [7]:
tau = []
state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED-4)
state = parse_state(state)
done = False
while not done:
    with torch.no_grad():
        probs = policy(state)
        action = torch.argmax(probs).item()
    next_state, reward, done, _, _ = env.step(action)
    next_state = parse_state(next_state)
    tau.append((state, action, reward))
    state = next_state
states, actions, rewards = zip(*tau)
states = torch.cat(states)
actions = torch.tensor([actions])
rewards = torch.tensor([rewards])
print(rewards)

tensor([[ 0.0000, -0.1004]], dtype=torch.float64)


In [21]:
def k_shot(policy, n_steps, seed, test=False):
  for e in range(n_steps):
    trajs = sample_rollouts(policy, env, 2, seed=seed, test=test)
    u = torch.stack([grad_U_tau(policy, tau, GAMMA) for tau in trajs]).mean(axis=0)
    g = u * torch.sqrt(2 * EPSILON / (u @ u))
    n = 0
    for i, p in enumerate(policy.parameters()):
      num_elements = p.numel()
      p.data += g[n:n+num_elements].view(p.shape)
      n += num_elements
    policy.zero_grad()

In [24]:
tau = []
state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED-4)
state = parse_state(state)
done = False
while not done:
    with torch.no_grad():
        probs = policy(state)
        action = torch.argmax(probs).item()
    next_state, reward, done, _, _ = env.step(action)
    next_state = parse_state(next_state)
    tau.append((state, action, reward))
    state = next_state
states, actions, rewards = zip(*tau)
states = torch.cat(states)
actions = torch.tensor([actions])
rewards = torch.tensor([rewards])
print(rewards)

tensor([[ 0.0000, -0.1004]], dtype=torch.float64)


In [27]:
original_params

OrderedDict([('fc1.weight',
              tensor([[nan, nan, nan, nan, nan, nan, nan, nan],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [-inf, -inf, -inf, -inf, -inf, nan, nan, inf],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [inf, inf, inf, inf, inf, nan, nan, -inf],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [-inf, -inf, -inf, -inf, -inf, nan, nan, inf],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [-inf, -inf, -inf, -inf, -inf, nan, nan, inf],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [-inf, -inf, -inf, -inf, -inf, nan, nan, inf],
                      [nan, nan, nan, nan, nan, nan, nan, nan],
                      [-inf, -inf, -inf, -inf, -inf, na